In [1]:
import glycosylator as gl

## Demo 1: Molecule

In [2]:
man9 = gl.Molecule.from_PDB('support/examples/man9.pdb', 1)

@> 246 atoms and 1 coordinate set(s) were parsed in 0.01s.


In [3]:
print(f'Number of torsional angles: {len(man9.torsionals)}')

Number of torsional angles: 236


In [4]:
i = 10
torsional = man9.torsionals[i]
print(f'Torsional Angle Before: {man9.measure_dihedral_angle(torsional)}')

man9.rotate_bond(torsional, 15, absolute=False)
print(f'Torsional Angle After: {man9.measure_dihedral_angle(torsional)}')

man9.rotate_bond(torsional, 45, absolute=True)
print(f'Torsional Angle absolute set: {man9.measure_dihedral_angle(torsional)}')

Torsional Angle Before: -85.3039552791497
Torsional Angle After: -70.30395527914972
Torsional Angle absolute set: 45.0000000000001


## Demo 2: Builder

In [5]:
import glycosylator as gl
import prody

In [6]:
topo_path = 'support/toppar_charmm/carbohydrates.rtf'
prm_path = 'support/toppar_charmm/carbohydrates.prm'
builder = gl.MoleculeBuilder(topo_path, prm_path)

In [7]:
nag = prody.parsePDB('support/examples/NAG.pdb')

@> 30 atoms and 1 coordinate set(s) were parsed in 0.00s.


In [8]:
nag = nag.select('not serial 4 9').toAtomGroup()

In [9]:
nag_complete, missing_atoms, bonds = builder.find_missing_atoms(nag)
builder.build_missing_atom_coord(nag_complete, missing_atoms, builder.Topology.topology['NAG']['IC'])

In [10]:
nag2, del_atoms, bonds2 = builder.build_from_patch(nag_complete, 2, 'NAG', 'G', 'G1', '14bb')

In [11]:
nag_complete = builder.delete_atoms(nag_complete, del_atoms)
nag2 = builder.delete_atoms(nag2, del_atoms)
nag_complete += nag2

In [12]:
builder.Topology.read_topology('support/topology/DUMMY.top')
man, del_atoms, bond_man = builder.build_from_DUMMY(1, 'MAN', 'G', '1G', 'DUMMY_MAN')

## Demo 3: Glycosylating Glycans

In [13]:
import glycosylator as gl
import nglview as nv
import prody

In [14]:
topo_path = 'support/toppar_charmm/carbohydrates.rtf'
prm_path = 'support/toppar_charmm/carbohydrates.prm'
g = gl.Glycosylator(
    topo_path,
    prm_path
)
g.load_glycan_topologies('support/topology/man9.top')
g.builder.Topology.read_topology('support/topology/DUMMY.top')

In [15]:
de_novo_man9 = g.glycosylate(None, 'MAN9')
print(f"Expected number of residue for de_novo_man9: {len(g.glycan_topologies['MAN9'])}")
print(f'de_novo_man9 has {len(de_novo_man9.residue_graph)} residues')

Expected number of residue for de_novo_man9: 11
de_novo_man9 has 11 residues


In [16]:
g.load_glycan_topologies('support/topology/mannose.top')

man9 = gl.Molecule.from_PDB('support/examples/man9.pdb', 1)
g.assign_patches(man9)
print(f"Expected number of residue for man9: {len(g.glycan_topologies['MAN9'])}")
print(f'man9 has {len(man9.residue_graph)} residues')

@> 246 atoms and 1 coordinate set(s) were parsed in 0.00s.


Expected number of residue for man9: 11
man9 has 11 residues


In [17]:

# trim man9 down to man6
man6 = g.glycosylate(man9, 'MAN6_1;3,2')
print(f"Expected number of residue for man6: {len(g.glycan_topologies['MAN6_1;3,2'])}")
print(f'man6 has {len(man6.residue_graph)} residues')

Expected number of residue for man6: 8
man6 has 8 residues


In [18]:
# extend man6 to man8
man8 = g.glycosylate(man6, 'MAN8_2;4,2')
print(f"Expected number of residue for man8: {len(g.glycan_topologies['MAN8_2;4,2'])}")
print(f'man8 has {len(man8.residue_graph)} residues')

Expected number of residue for man8: 10
man8 has 10 residues


In [19]:
patch_paths = [
    'NAG',
    'NAG C1 14bb',
    'BMA C1 14bb 14bb',
    'MAN C1 14bb 14bb 16ab',
    'MAN C1 14bb 14bb 16ab 16ab',
    'MAN C1 14bb 14bb 16ab 13ab',
    'MAN C1 14bb 14bb 16ab 13ab 12aa',
    'MAN C1 14bb 14bb 13ab'
]
manual_glycan_topo = gl.glycan_representations.GlycanTopology.from_patch_strings('custom_glycan', patch_paths)
manual_glycan = g.glycosylate(None, manual_glycan_topo)
print(f"Expected number of residue for manual_glycan: {len(manual_glycan_topo)}")
print(f'manual_glycan has {len(manual_glycan.residue_graph)} residues')
# save glycan if desired
# manual_glycan.write_PDB('manual_glycan.pdb')

Expected number of residue for manual_glycan: 8
manual_glycan has 8 residues


## Demo 4: Glycosylating Protein

In [20]:
import glycosylator as gl
import nglview as nv
import prody

In [21]:
topo_path = 'support/toppar_charmm/carbohydrates.rtf'
prm_path = 'support/toppar_charmm/carbohydrates.prm'
dummy_top = 'support/topology/DUMMY.top'
man_top = 'support/topology/mannose.top'
g = gl.Glycosylator(
    topo_path,
    prm_path
)
g.load_glycan_topologies(man_top)

In [22]:
hiv_env = prody.parsePDB('support/examples/env_4tvp.pdb')
# Residue number 625 on chain B
protein_residue = hiv_env[('B', 625)]

man8 = g.glycosylate(None, 'MAN8_3;3,2', protein_residue=protein_residue, protein_link_patch='NGLB')
g.identify_glycan(man8)

@> 5309 atoms and 1 coordinate set(s) were parsed in 0.06s.


'UnknownGlycan'

In [23]:
g.load_glycoprotein_from_AtomGroup(hiv_env)
# List of identified residues that are part of a sequon
sequons = g.find_linking_residues(sequon_pattern="(N[^P][S|T])")
# List of existing glycans, each containing an amino acid at the root
glycans = g.find_existing_glycans()
for glycan in glycans:
    root_res: prody.Residue = glycan.residue_graph.graph["root_residue"]
    g.assign_patches(glycan)
    glycan_name = g.identify_glycan(glycan)
    print(f"Root residue: {root_res.getResname()} {root_res.getResnum()} {root_res.getChid()} -> {glycan_name}")


Root residue: ASN 88 G -> MAN5_1;3,1
Root residue: ASN 133 G -> NAG1_0;0,0
Root residue: ASN 137 G -> MAN2_1;1,0
Root residue: ASN 156 G -> MAN3_1;2,0
Root residue: ASN 160 G -> NAG2_0;0,0
Root residue: ASN 197 G -> NAG2_0;0,0
Root residue: ASN 234 G -> NAG2_0;0,0
Root residue: ASN 262 G -> MAN4_2;2,0
Root residue: ASN 276 G -> NAG1_0;0,0
Root residue: ASN 295 G -> NAG2_0;0,0
Root residue: ASN 301 G -> NAG2_0;0,0
Root residue: ASN 332 G -> MAN8_3;3,2
Root residue: ASN 339 G -> NAG1_0;0,0
Root residue: ASN 355 G -> NAG1_0;0,0
Root residue: ASN 363 G -> NAG2_0;0,0
Root residue: ASN 386 G -> NAG2_0;0,0
Root residue: ASN 392 G -> NAG2_0;0,0
Root residue: ASN 448 G -> BMA1_0;0,0
Root residue: ASN 611 B -> NAG1_0;0,0
Root residue: ASN 618 B -> NAG1_0;0,0
Root residue: ASN 637 B -> NAG1_0;0,0


In [ ]:
protein_residue = hiv_env[('G', 262)]
original_glycan = g.find_glycan(protein_residue)

# TODO:
# this currently does not work, for some reason data structures go out of sync with the atom group:
# in glycan_topology the first residue is missing, so at some point a patch path cannot be found, even if the underlying atom_group seems to have all the residues.
new_glycan = g.glycosylate(original_glycan, 'MAN9_3;4,2', protein_residue, 'NGLB')
# export glycoprotein as pdb if desired
# g.write_glycoprotein_to_PDB('HIV_N262_man9.pdb')